In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L12-v2')
sentences = ["This is a sentence", "Here is another one"]
embeddings = model.encode(sentences)

/home/sann_htet/milvus-tutorial/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sann_htet/milvus-tutorial/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection

# Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# Define the schema for the collection
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embeddings.shape[1])
]
schema = CollectionSchema(fields, "Sentence embedding collection")

# Create the collection
collection = Collection(name="sentence_embeddings", schema=schema)

In [6]:
# Prepare the data for insertion
data = [
    embeddings.tolist()  # Embeddings
]

# Insert data into the collection
collection.insert(data)

(insert count: 2, delete count: 0, upsert count: 0, timestamp: 449886540275384322, success count: 2, err count: 0, cost: 0)

In [10]:
# Define the index parameters
INDEX_PARAM = {
    'metric_type':'L2',
    'index_type':"HNSW",
    'params':{'M': 8, 'efConstruction': 64}
}

# Create the index on the embedding field
collection.create_index(field_name="embedding", index_params=INDEX_PARAM)

# Load the collection into memory
collection.load()

In [11]:
# Generate an embedding for the query sentence
query_embedding = model.encode(["This is a query sentence"])

# Perform the search
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(query_embedding, "embedding", search_params, limit=10)

# Print the results
for result in results:
    for match in result:
        print(f"ID: {match.id}, Distance: {match.distance}")

ID: 449886475126310098, Distance: 0.5566655397415161
ID: 449886475126310101, Distance: 0.5566655397415161
ID: 449886475126310099, Distance: 1.696988582611084
ID: 449886475126310102, Distance: 1.696988582611084
